In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import dask.dataframe as dd
import dask.array as da
from dask import delayed
from dask.distributed import Client, SSHCluster,LocalCluster

In [2]:
# 用for循环提取文件数据
patent_list = []
for i in range(1980, 2023):
    patent_list.append(pd.read_csv(f'/home/bld/bld_data/pengliu/Graph data/Patent relation/patent_table2/cor_{i}.csv'))

In [3]:
patent_list[0].head()

,Unnamed: 0,10006,10057,10102,10103,10145,10153,10161,10162,10188,...,82799,82959,83300,83361,83433,83644,83687,83783,83855,84479
0,10006,1.000000,0.246089,0.232862,0.0,0.275006,0.534531,0.044146,0.074292,0.330563,...,0.130809,0.166123,0.091932,0.215800,0.000000,0.055374,0.000000,0.399584,0.000000,0.013844
1,10057,0.246089,1.000000,0.226486,0.0,0.206647,0.259065,0.089695,0.000000,0.071961,...,0.000000,0.000000,0.426941,0.097435,0.109150,0.112509,0.297670,0.000000,0.000000,0.000000
2,10102,0.232862,0.226486,1.000000,0.0,0.611208,0.253011,0.221638,0.000000,0.152761,...,0.051584,0.000000,0.064737,0.206839,0.198605,0.034120,0.090272,0.255327,0.000000,0.034120
3,10103,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.204124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,10145,0.275006,0.206647,0.611208,0.0,1.000000,0.314921,0.283026,0.051338,0.146846,...,0.151859,0.114795,0.010890,0.243567,0.133641,0.103315,0.106301,0.012272,0.022959,0.000000


In [4]:
# 检查patent_list中的数据是否有缺失值
for i in range(0, 43):
    print('data of year', 1980+i, 'has', patent_list[i].isna().sum().sum(), 'missing values')

data of year 1980 has 0 missing values
data of year 1981 has 0 missing values
data of year 1982 has 0 missing values
data of year 1983 has 0 missing values
data of year 1984 has 0 missing values
data of year 1985 has 0 missing values
data of year 1986 has 0 missing values
data of year 1987 has 0 missing values
data of year 1988 has 0 missing values
data of year 1989 has 0 missing values
data of year 1990 has 0 missing values
data of year 1991 has 0 missing values
data of year 1992 has 0 missing values
data of year 1993 has 0 missing values
data of year 1994 has 0 missing values
data of year 1995 has 0 missing values
data of year 1996 has 0 missing values
data of year 1997 has 0 missing values
data of year 1998 has 0 missing values
data of year 1999 has 0 missing values
data of year 2000 has 0 missing values
data of year 2001 has 0 missing values
data of year 2002 has 0 missing values
data of year 2003 has 0 missing values
data of year 2004 has 0 missing values
data of year 2005 has 0 m

In [5]:
# 检查patent_list数据形状
for i in range(0, 43):
    print('data of year', 1980+i, 'has shape', patent_list[i].shape)

data of year 1980 has shape (1853, 1854)
data of year 1981 has shape (1831, 1832)
data of year 1982 has shape (1819, 1820)
data of year 1983 has shape (1792, 1793)
data of year 1984 has shape (1792, 1793)
data of year 1985 has shape (1847, 1848)
data of year 1986 has shape (1870, 1871)
data of year 1987 has shape (1936, 1937)
data of year 1988 has shape (2058, 2059)
data of year 1989 has shape (2107, 2108)
data of year 1990 has shape (2126, 2127)
data of year 1991 has shape (2135, 2136)
data of year 1992 has shape (2122, 2123)
data of year 1993 has shape (2143, 2144)
data of year 1994 has shape (2141, 2142)
data of year 1995 has shape (2234, 2235)
data of year 1996 has shape (2323, 2324)
data of year 1997 has shape (2518, 2519)
data of year 1998 has shape (2696, 2697)
data of year 1999 has shape (2900, 2901)
data of year 2000 has shape (3020, 3021)
data of year 2001 has shape (3210, 3211)
data of year 2002 has shape (3302, 3303)
data of year 2003 has shape (3287, 3288)
data of year 200

In [6]:
# 将patent_list中的第一列作为索引
for i in range(0, 43):
    patent_list[i].set_index('Unnamed: 0', inplace=True)

In [7]:
patent_list[42].head()

,10051,10104,10107,10145,10180,10200,10253,10258,10259,10299,...,93295,93296,93301,93339,93340,93345,93354,93372,93401,93436
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
10051,1.000000,0.000230,0.005552,0.018321,0.904534,0.452267,0.000000,0.064121,0.000000,0.000000,...,0.000000,0.000000,0.532181,0.904534,0.000000,0.066058,0.000000,0.000000,0.000000,0.000000
10104,0.000230,1.000000,0.995138,0.522090,0.000000,0.001095,0.933689,0.000567,0.356797,0.297853,...,0.032862,0.039167,0.001568,0.000000,0.011073,0.013832,0.489315,0.000098,0.434699,0.004683
10107,0.005552,0.995138,1.000000,0.555181,0.005381,0.006637,0.899724,0.002153,0.430609,0.357693,...,0.044499,0.063845,0.008020,0.005381,0.025962,0.014521,0.558344,0.000309,0.449515,0.011525
10145,0.018321,0.522090,0.555181,1.000000,0.018123,0.141652,0.396573,0.128395,0.396573,0.362970,...,0.090664,0.693602,0.198408,0.018123,0.189945,0.016474,0.453160,0.030829,0.837202,0.116090
10180,0.904534,0.000000,0.005381,0.018123,1.000000,0.500000,0.000000,0.070888,0.000000,0.000000,...,0.000000,0.000000,0.588348,1.000000,0.000000,0.073030,0.000000,0.000000,0.000000,0.000000


In [8]:
# patent_list[]中1980-2022年数据转换成邻接表的形式
patent_adj_list = []
for i in range(0, 43):
    patent_adj_list.append(patent_list[i].stack().reset_index())
    patent_adj_list[i].columns = ['StartCompanyName', 'ConnCompanyName', 'Patent_similarity']

In [9]:
patent_adj_list[0].head()

,StartCompanyName,ConnCompanyName,Patent_similarity
0,10006,10006,1.000000
1,10006,10057,0.246089
2,10006,10102,0.232862
3,10006,10103,0.000000
4,10006,10145,0.275006


In [10]:
# 读取 .dta 文件
file_path = '/home/bld/bld_data/pengliu/CRSP_Compustat_Merged_Database_Linking_Table.dta'
CUSIP_PERMNO_data = pd.read_stata(file_path)

In [11]:
# 正确的列选择方式
selected_data = CUSIP_PERMNO_data[['cusip', 'LPERMNO','LPERMCO']]
selected_data

,cusip,LPERMNO,LPERMCO
0,000032102,25881.0,23369.0
1,000165100,10015.0,6398.0
2,000352104,10023.0,22159.0
3,000354100,10031.0,6672.0
4,000361105,54594.0,20000.0
...,...,...,...
31692,45339J105,15642.0,55395.0
31693,185053204,23514.0,59438.0
31694,37428A103,22205.0,58855.0
31695,74907L300,16161.0,55612.0


In [12]:
# 删除 LPERMNO 列中的重复值，保留第一次出现的值
selected_data = selected_data.drop_duplicates(subset='LPERMNO')
selected_data

,cusip,LPERMNO,LPERMCO
0,000032102,25881.0,23369.0
1,000165100,10015.0,6398.0
2,000352104,10023.0,22159.0
3,000354100,10031.0,6672.0
4,000361105,54594.0,20000.0
...,...,...,...
31689,Y4600W108,21069.0,58208.0
31690,583435102,20846.0,58137.0
31693,185053204,23514.0,59438.0
31694,37428A103,22205.0,58855.0


In [13]:
# 将 LPERMNO 作为索引
lpermno_to_cusip = selected_data.set_index('LPERMNO')['cusip']

In [14]:
# 将 patent_adj_list 中的 StartCompanyName 和 ConnCompanyName 中的 LPERMNO 转换成 cusip 代码
for i in range(0, 43):
    patent_adj_list[i]['StartCompanyName'] = patent_adj_list[i]['StartCompanyName'].astype(float).map(lpermno_to_cusip)
    patent_adj_list[i]['ConnCompanyName'] = patent_adj_list[i]['ConnCompanyName'].astype(float).map(lpermno_to_cusip)
    print('data of year', 1980+i, 'has shape', patent_adj_list[i].shape)

data of year 1980 has shape (3433609, 3)
data of year 1981 has shape (3352561, 3)
data of year 1982 has shape (3308761, 3)
data of year 1983 has shape (3211264, 3)
data of year 1984 has shape (3211264, 3)
data of year 1985 has shape (3411409, 3)
data of year 1986 has shape (3496900, 3)
data of year 1987 has shape (3748096, 3)
data of year 1988 has shape (4235364, 3)
data of year 1989 has shape (4439449, 3)
data of year 1990 has shape (4519876, 3)
data of year 1991 has shape (4558225, 3)
data of year 1992 has shape (4502884, 3)
data of year 1993 has shape (4592449, 3)
data of year 1994 has shape (4583881, 3)
data of year 1995 has shape (4990756, 3)
data of year 1996 has shape (5396329, 3)
data of year 1997 has shape (6340324, 3)
data of year 1998 has shape (7268416, 3)
data of year 1999 has shape (8410000, 3)
data of year 2000 has shape (9120400, 3)
data of year 2001 has shape (10304100, 3)
data of year 2002 has shape (10903204, 3)
data of year 2003 has shape (10804369, 3)
data of year 

In [15]:
patent_adj_list[42]

,StartCompanyName,ConnCompanyName,Patent_similarity
0,817335102,817335102,1.000000
1,817335102,68389X105,0.000230
2,817335102,594918104,0.005552
3,817335102,438516106,0.018321
4,817335102,009728106,0.904534
...,...,...,...
1763579,88160R101,192005106,0.000000
1763580,88160R101,60671Q104,0.024544
1763581,88160R101,25960R105,0.000000
1763582,88160R101,879455103,0.027824


In [16]:
patent_adj_list[42].isna().sum().sum()

7968

In [17]:
# 检查 patent_adj_list 中的缺失值
for i in range(0, 43):
    print('data of year', 1980+i, 'has', patent_adj_list[i].isna().sum().sum(), 'missing values')

data of year 1980 has 174182 missing values
data of year 1981 has 131832 missing values
data of year 1982 has 98226 missing values
data of year 1983 has 60928 missing values
data of year 1984 has 57344 missing values
data of year 1985 has 66492 missing values
data of year 1986 has 67320 missing values
data of year 1987 has 69696 missing values
data of year 1988 has 90552 missing values
data of year 1989 has 80066 missing values
data of year 1990 has 72284 missing values
data of year 1991 has 68320 missing values
data of year 1992 has 63660 missing values
data of year 1993 has 64290 missing values
data of year 1994 has 42820 missing values
data of year 1995 has 17872 missing values
data of year 1996 has 18584 missing values
data of year 1997 has 20144 missing values
data of year 1998 has 16176 missing values
data of year 1999 has 11600 missing values
data of year 2000 has 24160 missing values
data of year 2001 has 25680 missing values
data of year 2002 has 26416 missing values
data of y

In [18]:
# 删除 patent_adj_list 中的缺失值
for i in range(0, 43):
    patent_adj_list[i].dropna(inplace=True)
    print('data of year', 1980+i, 'has shape', patent_adj_list[i].shape)

data of year 1980 has shape (3261636, 3)
data of year 1981 has shape (3222025, 3)
data of year 1982 has shape (3211264, 3)
data of year 1983 has shape (3150625, 3)
data of year 1984 has shape (3154176, 3)
data of year 1985 has shape (3345241, 3)
data of year 1986 has shape (3429904, 3)
data of year 1987 has shape (3678724, 3)
data of year 1988 has shape (4145296, 3)
data of year 1989 has shape (4359744, 3)
data of year 1990 has shape (4447881, 3)
data of year 1991 has shape (4490161, 3)
data of year 1992 has shape (4439449, 3)
data of year 1993 has shape (4528384, 3)
data of year 1994 has shape (4541161, 3)
data of year 1995 has shape (4972900, 3)
data of year 1996 has shape (5377761, 3)
data of year 1997 has shape (6320196, 3)
data of year 1998 has shape (7252249, 3)
data of year 1999 has shape (8398404, 3)
data of year 2000 has shape (9096256, 3)
data of year 2001 has shape (10278436, 3)
data of year 2002 has shape (10876804, 3)
data of year 2003 has shape (10778089, 3)
data of year 

In [19]:
# patent_adj_list[i]在第0列增加一列，列名为'Date'，值为1980+i
for i in range(0, 43):
    patent_adj_list[i].insert(0, 'Date', 1980+i)

In [20]:
patent_adj_list[0]

,Date,StartCompanyName,ConnCompanyName,Patent_similarity
0,1980,00099V004,00099V004,1.000000
1,1980,00099V004,004626107,0.246089
2,1980,00099V004,009292103,0.232862
3,1980,00099V004,000874107,0.000000
4,1980,00099V004,438516106,0.275006
...,...,...,...,...
3433603,1980,205831308,979438108,0.000000
3433605,1980,205831308,983085101,0.000000
3433606,1980,205831308,984123109,0.000000
3433607,1980,205831308,984900001,0.000000


In [21]:
patent_adj_list[42]

,Date,StartCompanyName,ConnCompanyName,Patent_similarity
0,2022,817335102,817335102,1.000000
1,2022,817335102,68389X105,0.000230
2,2022,817335102,594918104,0.005552
3,2022,817335102,438516106,0.018321
4,2022,817335102,009728106,0.904534
...,...,...,...,...
1763579,2022,88160R101,192005106,0.000000
1763580,2022,88160R101,60671Q104,0.024544
1763581,2022,88160R101,25960R105,0.000000
1763582,2022,88160R101,879455103,0.027824


In [22]:
# 将 patent_adj_list[]中的数据逐个保存到csv文件中
# /home/bld/bld_data/pengliu/Graph data/Patent relation/final_data_2
for i in range(0, 43):
    patent_adj_list[i].to_csv(f'/home/bld/bld_data/pengliu/Graph data/Patent relation/final_data_3/patent_{1980+i}.csv', index=False)